In [ ]:
!pip install openai

In [ ]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import datetime
from openai import OpenAI

# Fetch email & summarize it with GPT-4

In [ ]:
# Initialize OpenAI client
client = OpenAI(
    api_key=openai_api_key
)

# Create an IMAP4 class with SSL
mail = imaplib.IMAP4_SSL("imap.gmail.com")
# Authenticate
mail.login(username, password)

# Select the Inbox
mail.select("inbox")

# Define the date from which you want to search emails
days_limit = 30
v_start_date = (datetime.date.today() - datetime.timedelta(days=days_limit)).strftime("%d-%b-%Y")

# Search for emails in the Promotions category since the specified date
status, messages = mail.uid('search', None, f'SINCE {v_start_date} X-GM-RAW "category:promotions"')

# Convert messages to a list of email UIDs
email_uids = messages[0].split()

# Fetch the most recent email
if email_uids:
    most_recent_email_uid = email_uids[-1]

    # Fetch the email by UID
    status, msg_data = mail.uid('fetch', most_recent_email_uid, "(RFC822)")

    # Parse the email content
    for response_part in msg_data:
        if isinstance(response_part, tuple):
            msg = email.message_from_bytes(response_part[1])
            # Decode the email subject
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding if encoding else "utf-8")
            # Decode the email sender
            from_ = msg.get("From")
            print("Subject:", subject)
            print("From:", from_)

            # Initialize variable to store email body
            email_body = ""

            # If the email message is multipart
            if msg.is_multipart():
                # Iterate over email parts
                for part in msg.walk():
                    # Extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))

                    try:
                        # Get the email body
                        body = part.get_payload(decode=True).decode()
                    except:
                        pass
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        email_body = body
                    elif content_type == "text/html":
                        email_body = body
            else:
                # Extract content type of email
                content_type = msg.get_content_type()

                # Get the email body
                body = msg.get_payload(decode=True).decode()
                if content_type == "text/plain":
                    email_body = body

            # Summarize the email content using OpenAI API
            if email_body:
                response = client.chat.completions.create(
                    model="gpt-4",
                    messages=[
                        {"role": "user", "content": f"Summarize the following email content:\n\n{email_body[:8000]}"}
                    ]
                )
                summary = response.choices[0].message.content
                print("Summary:", summary)

# Close the connection and logout
mail.close()
mail.logout()

Subject: Welcome To The Players League 
From: =?utf-8?b?4pmfQ2hlc3MuY29t?= <hello@chess.com>
Summary: The email is from Chess.com encouraging its members to earn trophies and climb the ranks of the site.


('BYE', [b'LOGOUT Requested'])